In [16]:
import openai
import re

In [12]:
input_file = "lectures/business_cycle.md"
print(openai.beta.assistants.list())
assistant_cn_id = 'asst_zjzyGwEZ1rVuJYWNQk6nzQTA'

SyncCursorPage[Assistant](data=[Assistant(id='asst_qKEHiaBYlDSpTpNKCVjhVllL', created_at=1721564291, description=None, instructions=None, metadata={}, model='gpt-4o', name='Translator', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0), Assistant(id='asst_kWnZVKQEHRY1Db6ezdbKnRIy', created_at=1721564130, description=None, instructions='Translate the following lecture into Chinese', metadata={}, model='gpt-4o', name='Translator', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0), Assistant(id='asst_zjzyGwEZ1rVuJYWNQk6nzQTA', created_at=1721563447, description=None, instructions='You are working on a translation project migrating QuantEcon lectures to Chinese. Your task is to maintain the myst markdown syntax and headers while translating the main text and code comment to simplified Ch

In [54]:
def split_text(content, chunk_size=3000):
    chunks = []
    start = 0
    while start < len(content):
        end = start + chunk_size

        # If we are at the end of the content, just append the rest
        if end >= len(content):
            chunks.append(content[start:])
            break

        # Find the nearest line break before the chunk size
        next_line_break = content.rfind('\n', start, end)
        if next_line_break == -1:
            # If no line break is found within the chunk size, extend to the end
            next_line_break = end

        # Check if a code cell starts within the chunk
        code_cell_start = content.find('```{code-cell}', start, next_line_break)
        if code_cell_start != -1:
            # If a code cell starts, find its end
            code_cell_end = content.find('```', code_cell_start + 14)
            if code_cell_end != -1:
                # Move the end to the end of the code cell
                next_line_break = content.find('\n', code_cell_end) + 1

        chunks.append(content[start:next_line_break].strip())
        start = next_line_break

    return chunks
# Example usage
content = """Your text content here. Make sure it's long enough to be split into multiple chunks. Each chunk will end at a complete line.
This is another line. Make sure to handle new lines properly.
And yet another line to test the function.

```{code-cell} python3
print("This is a code cell")
print("It should not be broken across chunks")
```
"""

chunks = split_text_at_line_end(content, chunk_size=150)
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}:\n{chunk}\n")

Chunk 1:
Your text content here. Make sure it's long enough to be split into multiple chunks. Each chunk will end at a complete line.
This is another line. Make sure to handle new lines properly.

Chunk 2:
And yet another line to test the function.

```{code-cell} python3
print("This is a code cell")
print("It should not be broken across chunks")
```




In [55]:
# Initialize the OpenAI client
client = openai.OpenAI()

# Read the content of the input markdown file
with open(input_file, 'r', encoding='utf-8') as file:
    content = file.read()

# Split the content into chunks (e.g., 3000 characters each)
chunks = split_text(content, chunk_size=3000)

# Initialize the OpenAI client and thread
thread = client.beta.threads.create()

translated_content = ""

for chunk in chunks:
    # Create and poll the run for each chunk
    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread.id,
        assistant_id=assistant_cn_id,
        instructions="Please translate the following content into simplified Chinese. Maintain all the markdown syntax and directives unchanged. Only translate text and code comments Give the results directly without system messages: " + chunk
    )
    
    if run.status == 'completed': 
        messages = client.beta.threads.messages.list(
            thread_id=thread.id
        )
        translated_content += '\n\n' + messages.data[0].content[0].text.value
        print(translated_content)
    else:
        print(f"Translation failed for chunk: {chunk[:50]}... Status: {run.status}")
        continue

# Create the output file name
output_file = input_file.replace('.md', '_cn.md')

# Write the translated content to the new markdown file
with open(output_file, 'w', encoding='utf-8') as file:
    file.write(translated_content)

print(f"Translated content has been saved to {output_file}")




---
jupytext:
  text_representation:
    extension: .md
    format_name: myst
    format_version: 0.13
    jupytext_version: 1.14.5
kernelspec:
  display_name: Python 3 (ipykernel)
  language: python
  name: python3
---

# 经济周期

## 概述

在本讲座中，我们将回顾经济周期的一些经验方面。

经济周期是经济活动随时间波动的现象。

这些包括扩张（也称为繁荣）和收缩（也称为衰退）。

为了我们的研究，我们将使用来自[世界银行](https://documents.worldbank.org/en/publication/documents-reports/api)和[FRED](https://fred.stlouisfed.org/)的经济指标。

除了由Anaconda已安装的软件包外，本讲座还需要

```{code-cell} ipython3
:tags: [hide-output]

!pip install wbgapi
!pip install pandas-datareader
```
图像输入功能：已启用


---
jupytext:
  text_representation:
    extension: .md
    format_name: myst
    format_version: 0.13
    jupytext_version: 1.14.5
kernelspec:
  display_name: Python 3 (ipykernel)
  language: python
  name: python3
---

# 经济周期

## 概述

在本讲座中，我们将回顾经济周期的一些经验方面。

经济周期是经济活动随时间波动的现象。

这些包括扩张（也称为繁荣）和收缩（也称为衰退）。

为了我们的研究，我们将使用来自[世界银行](https://documents.worldbank.org/en/publication/documents-reports/api)和[FRED](https:/

In [10]:
messages.data[0].content[0].text.value

'```{solution-end}\n```\n\n```{exercise}\n:label: ar1p_ex3\n\n在讲座中，我们讨论了以下事实：对于 $AR(1)$ 过程\n\n$$\nX_{t+1} = a X_t + b + c W_{t+1}\n$$\n\n其中 $\\{ W_t \\}$ 是标准正态独立同分布，\n\n$$\n\\psi_t = N(\\mu, s^2) \\implies \\psi_{t+1}\n= N(a \\mu + b, a^2 s^2 + c^2)\n$$\n\n通过模拟确认这一点，至少近似地。设定\n\n- $a = 0.9$\n- $b = 0.0$\n- $c = 0.1$\n- $\\mu = -3$\n- $s = 0.2$\n\n首先，使用上述描述的真实分布绘制 $\\psi_t$ 和 $\\psi_{t+1}$。\n\n其次，如下所示在同一图上绘制 $\\psi_{t+1}$（使用不同颜色）：\n\n1. 从 $N(\\mu, s^2)$ 分布中生成 $n$ 个 $X_t$ 的抽样值\n2. 使用规则 $X_{t+1} = a X_t + b + c W_{t+1}$ 更新这些抽样值\n3. 使用核密度估计从结果 $X_{t+1}$ 样本生成密度估计\n\n尝试使用 $n=2000$，确认基于模拟的 $\\psi_{t+1}$ 估计确实收敛于理论分布。\n```\n\n```{solution-start} ar1p_ex3\n:class: dropdown\n```\n\n这里是我们的解决方案：\n\n```{code-cell} ipython3\na = 0.9\nb = 0.0\nc = 0.1\nμ = -3\ns = 0.2\n```\n\n```{code-cell} ipython3\nμ_next = a * μ + b\ns_next = np.sqrt(a**2 * s**2 + c**2)\n```\n\n```{code-cell} ipython3\nψ = lambda x: K((x - μ) / s)\nψ_next = lambda x: K((x - μ_next) / s_next)\n```\n\n```{code-cell} ipython3\nψ = norm